In [ ]:
import rtsvg
rt = rtsvg.RACETrack()

class XWords(object):
    #
    # Constructor
    #
    def __init__(self, rt_self, entries_file, geometries_file, blockers_file, cell_w=32, cell_h=32):
        self.rt_self         = rt_self
        self.entries_file    = entries_file
        self.geometries_file = geometries_file
        self.blockers_file   = blockers_file
        self.cell_w          = cell_w
        self.cell_h          = cell_h
        with open(self.entries_file) as f:
            self.entries    = eval(f.read())
        with open(self.geometries_file) as f:
            self.geometries = eval(f.read())
        with open(self.blockers_file) as f:
            self.blockers   = eval(f.read())
        
        # Determine the number of tiles
        self.x_tiles, self.y_tiles = 1, 1
        for blocker in self.blockers:
            self.x_tiles = max(self.x_tiles, blocker[0] + 1)
            self.y_tiles = max(self.y_tiles, blocker[1] + 1)
        for geometry in self.geometries:
            self.x_tiles = max(self.x_tiles, geometry[1] + 1)
            self.y_tiles = max(self.y_tiles, geometry[2] + 1)
        self.w = self.x_tiles * cell_w
        self.h = self.y_tiles * cell_h

        class XWCell(object):
            def __init__(self, xi, yi):
                self.xi, self.yi    = xi, yi
                self.__cluenum__    = None
                self.__is_blocker__ = False
                self.__clues__      = {}
            def setClueNumber(self, cluenum): 
                self.__cluenum__ = cluenum
                if self.__is_blocker__:
                    raise Exception(f'XWCell.setCluenNumber() -- blocker already set {self.__is_blocker__} ({self.xi},{self.yi})')
            def isBlocker(self): 
                return self.__is_blocker__
            def setBlocker(self): 
                self.__is_blocker__ = True
                if self.__cluenum__ is not None:
                    raise Exception(f'XWCell.setBlocker() -- cluenum already set {self.__cluenum__} ({self.xi},{self.yi})')
            def addClue(self, cluenum, clue, orientation):
                if self.__cluenum__ != cluenum:
                    raise Exception(f'XWCell.addClue() -- cluenum mismatch {self.__cluenum__} != {cluenum} ({self.xi},{self.yi})')
                if orientation in self.__clues__:
                    raise Exception(f'XWCell.addClue() -- orientation already set {orientation} ({self.xi},{self.yi})')
                self.__clues__[orientation] = clue

        # Create a two dimensions structure that captures the state of each cell
        self.cells = [] # self.cells[yi][xi]
        for yi in range(self.y_tiles):
            _row_ = []
            for xi in range(self.x_tiles): _row_.append(XWCell(xi, yi))
            self.cells.append(_row_)
        # ... fill the cells in with information from the other structures
        self.cluenum_to_cell = {}
        for geometry in self.geometries:
            cluenum, xi, yi               = geometry
            _cell_                        = self.cells[yi][xi]
            _cell_.setClueNumber(cluenum)
            self.cluenum_to_cell[cluenum] = _cell_
        for cluenum_orientation in self.entries:
            cluenum, orientation          = cluenum_orientation
            clue                          = self.entries[cluenum_orientation]
            _cell_                        = self.cluenum_to_cell[cluenum]
            _cell_.addClue(cluenum, clue, orientation)
        for blocker in self.blockers:
            xi, yi                        = blocker
            _cell_                        = self.cells[yi][xi]
            _cell_.setBlocker()

    #
    # _repr_svg_() - return an svg representation of the crossword puzzle
    #
    def _repr_svg_(self):
        svg = [f'<svg x="0" y="0" width="{self.w}" height="{self.h}" viewBox="-5 -5 {self.w+10} {self.h+10}">']
        for x_tile in range(self.x_tiles+1):
            svg.append(f'<line x1="{x_tile*self.cell_w}" y1="0" x2="{x_tile*self.cell_w}" y2="{self.h}" stroke="black" stroke-width="0.25" />')
        for y_tile in range(self.x_tiles+1):
            svg.append(f'<line x1="0" y1="{y_tile*self.cell_h}" x2="{self.w}" y2="{y_tile*self.cell_h}" stroke="black" stroke-width="0.25" />')
        for blocker in self.blockers:
            svg.append(f'<rect x="{blocker[0]*self.cell_w}" y="{blocker[1]*self.cell_h}" width="{self.cell_w}" height="{self.cell_h}" fill="black" />')
        svg.append('</svg>')
        return ''.join(svg)

import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
xwords = XWords(rt, entries_file, geometries_file, blockers_file)
xwords